In [1]:
import sys
sys.path.append('../')

In [2]:
from Proximity import prox_model, Proximity_Indicator
from brix import Indicator, Handler, Grid_maker
import US_State
import datetime
import geopandas as gpd
import json
import requests

# In this example, we will:
    - initialise a CityScope table using brix
    - download some census data filterrd to the area surrounding the grid
    - create and deploy a proximity indicator

# Initialise the table

In [3]:
table_name='epa'
state_fips=6
geom_type='block'

In [4]:
lat,lon = 37.47, -122.15634055
G = Grid_maker(table_name, lat, lon, 
               cell_size=55, nrows=50, ncols=50, rotation=30)

https://cityio.media.mit.edu/api/tables/list/ Attempt: 0
Table epa already exists
Calculating initial coordinates of each cell
Defining properties and headers


/opt/anaconda3/envs/prox/lib/python3.10/site-packages/brix/grid_maker.py:82: UserWarning: Table epa already exists
  warn(f'Table {self.table_name} already exists')


In [20]:
geogrid=G.get_grid_geojson()

### Define the cell types

In [22]:
types=json.load(open('../data/default_types.json'))
geogrid['properties']['types']=types

### Initialise the grid cells all to None type

In [23]:
for i in range(len(geogrid['features'])):
    geogrid['features'][i]['properties']['name']='None'
    geogrid['features'][i]['properties']['color']=types['None']['color']

### Set some cells to be interactive based on overlap with a boundary

In [24]:
interactive_area=gpd.read_file(
"""
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -122.1340012550354,
              37.48372979560496
            ],
            [
              -122.13342189788817,
              37.482384632757686
            ],
            [
              -122.13419437408449,
              37.47972829068993
            ],
            [
              -122.13535308837889,
              37.47724214141305
            ],
            [
              -122.13432312011719,
              37.47727619881123
            ],
            [
              -122.13428020477295,
              37.474108794382126
            ],
            [
              -122.13052511215209,
              37.47436423519954
            ],
            [
              -122.13061094284058,
              37.475692513376735
            ],
            [
              -122.12966680526732,
              37.47575211504922
            ],
            [
              -122.12964534759521,
              37.47693562413104
            ],
            [
              -122.1289587020874,
              37.47696968166888
            ],
            [
              -122.13181257247923,
              37.47937069896909
            ],
            [
              -122.129967212677,
              37.48372979560496
            ],
            [
              -122.1340012550354,
              37.48372979560496
            ]
          ]
        ]
      }
    }
  ]
}
""")

In [25]:
overlap_ids=gpd.overlay(interactive_area,
                        gpd.read_file(json.dumps(geogrid)), 
                        'intersection')['id'].values

In [30]:
for i in range(len(geogrid['features'])):
    if i in overlap_ids:
        geogrid['features'][i]['properties']['interactive']=True
    else:
        geogrid['features'][i]['properties']['interactive']=False
        geogrid['features'][i]['properties']['color']=[50,50,50,50]

### Send the grid data to CityIO

In [31]:
headers = {'Content-Type': 'application/json'}
url='https://cityio.media.mit.edu/api/table/{}'.format(table_name)
r = requests.post(url+'/GEOGRID', data = json.dumps(geogrid), headers=headers)

For the change to be reflected on the front-end, we need to update the current grid state based on the changes to the GEOGRID

In [32]:
G.reset_geogrid_data()

https://cityio.media.mit.edu/api/table/epa/GEOGRID/ Attempt: 0
Proceeding without reviewing GEOGRIDDATA
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
GEOGRIDDATA successfully updated: wTFjHvG8e6CSSBeuuwjArjh2fcP5GUNMpwkeTgRWdif


The front-end is now ready but there are no analysis modules running yet so nothing happens when we interact with the grid

# Set up a back end module

### Create the Handler

In [4]:
H=Handler(table_name)

https://cityio.media.mit.edu/api/tables/list/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0


To intialise the proximity indicator, we need geodataframes of (i) the static places/POIs and (ii) the GEOGRID

In [5]:
geogrid=H.get_geogrid_data(include_geometries=True).as_df()

https://cityio.media.mit.edu/api/table/epa/GEOGRIDDATA/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/GEOGRID/ Attempt: 0


/opt/anaconda3/envs/prox/lib/python3.10/site-packages/brix/classes.py:981: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  geos = gpd.GeoDataFrame(geos.drop('geometry',1),geometry=geos['geometry'].apply(lambda x: shape(x))) # no crs to avoid warning
/opt/anaconda3/envs/prox/lib/python3.10/site-packages/brix/classes.py:251: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  geogrid_data = gpd.GeoDataFrame(geogrid_data.drop('geometry',1),geometry=geogrid_data['geometry'].apply(lambda x: shape(x)),crs='EPSG:4326')


### Get some baseline places data by downloading geometres and demographic data from census

In [ ]:
st=US_State.US_State(state_fips=state_fips, year=2019, geom_type=geom_type)
st.get_geometry()
st.get_lodes_data( include=['wac', 'rac'])
st.add_lodes_cols_to_shape()

Getting geometry (block) for state: California


Subset the geometry by distance from the grid to reduce the size 

In [ ]:
st.subset_geom_by_distance(centre_x_y=[lon, lat], 
                                   radius=5000, name='model_area')
zones_subset=st.return_geometry('model_area')
zones_subset.explore()

### Define the proximity targets
- column is the column of the GeoDataFrame corresponding to this POI. The column can be binary (indicating whther the row is one these POIs) or numerical (indicating the capacity of the POI contained in this place.
- 'from' represents the relevant source location for this POI. eg. if we want a POI (eg. employment) to be located close to where people live, then 'from' should be the column representing residential population.
- 'max' is used in the linear normalisation of indicators. If the proximity result is equal to or greater than the max, the score will be 1.

In [ ]:
target_settings={'housing': {'column': 'res_total','max': 100000, 'from': 'emp_total'},
                 'healthcare': {'column': 'emp_naics_62','max': 10000, 'from': 'res_total'},
                 'jobs': {'column': 'emp_total','max': 100000, 'from': 'res_total'},
                'shopping': {'column': 'emp_naics_44-45','max': 10000, 'from': 'res_total'},
                'education': {'column': 'emp_naics_61','max': 10000, 'from': 'res_total'},
                'entertainment': {'column': 'emp_naics_71','max': 10000, 'from': 'res_total'}}

### Create the proximity indicator.
- 'max_dist' is the network search radius in meters. eg. for 15 minutes walking accessibility, use max_dist=1200 (assuming average espeed of 4.8km/hr)

In [ ]:
p=Proximity_Indicator(static_places=zones_subset, geogrid=geogrid, 
                      max_dist=2400, target_settings=target_settings)

In [ ]:
H.add_indicator(p)
H.listen()